In [1]:
import numpy as np
import pandas as pd
import os
import re

# 1.1	Creating dictionary of Positive and Negative words

In [2]:
df = pd.read_csv('D:/Black Coffer/LoughranMcDonald_MasterDictionary_2018.csv')
df

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,277,1.480000e-08,1.240000e-08,3.560000e-06,84,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.600000e-10,9.730000e-12,9.860000e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.280000e-10,1.390000e-10,6.230000e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,12,6.410000e-10,3.160000e-10,9.380000e-08,12,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,7250,3.870000e-07,3.680000e-07,3.370000e-05,914,0,0,0,0,0,0,0,0,0,0,3,12of12inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86481,ZYGOTE,86482,46,2.460000e-09,9.690000e-10,2.060000e-07,31,0,0,0,0,0,0,0,0,0,0,2,12of12inf
86482,ZYGOTES,86483,1,5.340000e-11,2.040000e-11,2.070000e-08,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
86483,ZYGOTIC,86484,0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,0,3,12of12inf
86484,ZYMURGIES,86485,0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [3]:
df1 = df[['Word','Positive']]
df2 = df1[df1.Positive > 0]
df3 = df2["Word"]
df3.to_csv('D:/Black Coffer/Positivewords.csv', header = None, index = None)
with open('D:/Black Coffer/Positivewords.csv', 'r') as inp, open('D:/Black Coffer/Positivewords.txt', 'w') as out:
    for line in inp:
        out.write(line)

In [4]:
df4 = df[['Word','Negative']]
df5 = df4[df4.Negative > 0]
df6 = df5["Word"]
df6.to_csv('D:/Black Coffer/Negativewords.csv', header = None, index = None)
with open('D:/Black Coffer/Negativewords.csv', 'r') as inp, open('D:/Black Coffer/Negativewords.txt', 'w') as out:
    for line in inp:
        out.write(line)

# 1.2	Cleaning using Stop Words Lists

In [5]:
# Loading stopwords dictionary for removing stop words
stopwordsfile = 'D:/Black Coffer/StopWords Lists/StopWords_Auditor.txt'
stopwordsfile = 'D:/Black Coffer/StopWords Lists/StopWords_Currencies.txt'
stopwordsfile = 'D:/Black Coffer/StopWords Lists/StopWords_DatesandNumbers.txt'
stopwordsfile = 'D:/Black Coffer/StopWords Lists/StopWords_Generic.txt'
stopwordsfile = 'D:/Black Coffer/StopWords Lists/StopWords_GenericLong.txt'
stopwordsfile = 'D:/Black Coffer/StopWords Lists/StopWords_Geographic.txt'
stopwordsfile = 'D:/Black Coffer/StopWords Lists/StopWords_Names.txt'
with open(stopwordsfile ,'r') as stop_words:
    stopwords = stop_words.read().lower()
stopwordlist = stopwords.split('\n')
stopwordlist[-1:] = []

# 1.3 List of tokens using the nltk tokenize module

In [6]:
from nltk.tokenize import RegexpTokenizer, sent_tokenize

In [7]:
def tokenizer(text):
    text = str(text).lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopwordlist, tokens))
    return filtered_words

### Loading Positive Words

In [8]:
posfile = 'D:/Black Coffer/Positivewords.txt'
with open(posfile,'r') as poswordfile:
    positivewords=poswordfile.read().lower()
positivewordlist=positivewords.split('\n')

### Loading Negative Words

In [9]:
negFile = 'D:/Black Coffer/Negativewords.txt'
with open(negFile ,'r') as negwordfile:
    negativeword = negwordfile.read().lower()
negativewordlist = negativeword.split('\n')

# Section 1.1: Positive score, Negative score, Polarity score

### Calculating Positive Score

In [10]:
def positive_score(text):
    numpositiveword = 0
    tokens = tokenizer(text)
    for word in tokens:
        if word in positivewordlist:
            numpositiveword  += 1
    
    posscore = numpositiveword
    return posscore

### Calculating Negative Score

In [11]:
def negative_score(text):
    numnegativeword = 0
    Token = tokenizer(text)
    for word in Token:
        if word in negativewordlist:
            numnegativeword -= 1
    negscore = numnegativeword 
    nscore = negscore * -1
    return nscore

### Calculating Polarity Score

##### Formula for Polarity Score = (positivescore - negativescore) / ((positivescore + negativescore) + 0.000001)

In [12]:
def polarity_score(positivescore, negativescore):
    pol_score = (positivescore - negativescore) / ((positivescore + negativescore) + 0.000001)
    return pol_score

# Section 2: Average Sentence Length, Percentage of Complex Words, Fog Index

### Calculating Average Sentence Length

##### Formula for Average Sentence Length = the number of words / the number of sentences

In [13]:
def average_sentence_length(text):
    text = str(text).lower()
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalwordcount = len(tokens)
    totalsentences = len(sentence_list)
    average_sent = 0
    if totalsentences != 0:
        average_sent = totalwordcount / totalsentences
    
    average_sent_length= average_sent
    
    return round(average_sent_length)

### Calculating Percentage of Complex Words

##### Formula for Percentage of Complex words = the number of complex words / the number of words

In [14]:
def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexword = 0
    percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexword += 1
    if len(tokens) != 0:
        percentage = complexword/len(tokens)
    
    return percentage

### Calculating Fog Index

##### Formula for Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

In [15]:
def fog_index(averagesentencelength, percentagecomplexword):
    fogindex = 0.4 * (averagesentencelength + percentagecomplexword)
    return fogindex

# Section 4: Complex Word Count

### Counting Complex Word

In [16]:
def complex_word_count(text):
    tokens = tokenizer(text)
    complexwordcount = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexwordcount += 1
    return complexwordcount

# Section 5: Word count

### Counting Word

In [17]:
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

# Calculating Variables from Uncertainty Dictionary

### Converting 'xlsx' to 'csv' to 'txt' file

In [18]:
read_file = pd.read_excel (r'D:/Black Coffer/uncertainty_dictionary.xlsx')
read_file.to_csv (r'D:/Black Coffer/uncertainty_dictionary.csv', index = None, header = None)
with open('D:/Black Coffer/uncertainty_dictionary.csv', 'r') as inp, open('D:/Black Coffer/uncertainty_dictionary.txt', 'w') as out:
    for line in inp:
        out.write(line)

### Calculating Uncertainty Score

In [19]:
uncertainty_dictionaryFile = 'D:/Black Coffer/uncertainty_dictionary.txt'
with open(uncertainty_dictionaryFile ,'r') as uncertain_dict:
    uncertainDict = uncertain_dict.read().lower()
uncertainDictionary = uncertainDict.split('\n')

def uncertainty_score(text):
    uncertainwordnum =0
    tokens = tokenizer(text)
    for word in tokens:
        if word in uncertainDictionary:
            uncertainwordnum +=1
    Uncertainityscore = uncertainwordnum 
    
    return Uncertainityscore

# Calculating Variables from Constraining Dictionary

### Converting 'xlsx' to 'csv' to 'txt' file

In [20]:
read_file = pd.read_excel (r'D:/Black Coffer/constraining_dictionary.xlsx')
read_file.to_csv (r'D:/Black Coffer/constraining_dictionary.csv', index = None, header = None)
with open('D:/Black Coffer/constraining_dictionary.csv', 'r') as inp, open('D:/Black Coffer/constraining_dictionary.txt', 'w') as out:
    for line in inp:
        out.write(line)

### Calculating Constraining Score

In [21]:
constraining_dictionaryFile = 'D:/Black Coffer/constraining_dictionary.txt'
with open(constraining_dictionaryFile ,'r') as constraining_dict:
    constrainDict = constraining_dict.read().lower()
constrainDictionary = constrainDict.split('\n')

def constraining_score(text):
    constrainwordnum = 0
    tokens = tokenizer(text)
    for word in tokens:
        if word in constrainDictionary:
            constrainwordnum +=1
    Constrainscore = constrainwordnum 
    
    return Constrainscore

# 2     Extraction of Sections from SEC/EDGAR Financial Reports 

In [22]:
df7 = pd.read_excel('D:/Black Coffer/cik_list.xlsx')
max_row, max_col = df7.shape
print(max_row)

pd.set_option('display.max_colwidth',100) # to display full text in column
df7

152


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt
...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,edgar/data/12239/0001104659-07-024804.txt
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,edgar/data/12239/0001104659-07-040463.txt
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,edgar/data/12239/0001104659-07-041441.txt
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,edgar/data/12239/0001104659-07-042333.txt


#### Adding the initial structure of the link in secfname column

In [23]:
link = 'https://www.sec.gov/Archives/'
df7.SECFNAME = link+df7.SECFNAME
df7.SECFNAME.head()

0    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt
1    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt
2    https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt
3    https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt
4    https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt
Name: SECFNAME, dtype: object

In [24]:
df7

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt
...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-024804.txt
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-040463.txt
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-041441.txt
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-042333.txt


#### Saving of Data Frame to new '.xlsx' file

In [25]:
df7.to_excel('D:/Black Coffer/cik_list_add.xlsx', index = None)

#### Data Extraction from 'cik_list_add.xlsx' after adding link to contents of secfname column

In [26]:
df8 = pd.read_excel('D:/Black Coffer/cik_list_add.xlsx',)
max_row, max_col = df8.shape
print(max_row)

pd.set_option('display.max_colwidth',100) # to display full text in column
df8

152


,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt
...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-024804.txt
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-040463.txt
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-041441.txt
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-042333.txt


### Text extraction patterns

In [27]:
mda_re = r"item[^a-zA-Z\n]*\d\s*\.\s*management\'s discussion and analysis.*?^\s*item[^a-zA-Z\n]*\d\s*\.*"
qqdmr_re = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Quantitative and Qualitative Disclosures about " \
        r"Market Risk.*?^\s*item\s*\d\s*" 
rf_re = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Risk Factors.*?^\s*item\s*\d\s*"

#### Function for Extraction of text from "Management's Discussion and Analysis" Report

In [28]:
import requests
def get_mda(url):
        
    source = requests.get(f'{url}').text.lower()
    mda = re.findall(mda_re, source, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    if mda:
        return ''.join(mda)
    else:
        return 0
    
df8['mda_text'] = df8.SECFNAME.apply(get_mda)

#### Function for Extraction of text from "Quantitative and Qualitative Disclosures about Market Risk" Report

In [29]:
def get_qqdmr(url):
        
    source = requests.get(f'{url}').text.lower()
    qqdmr = re.findall(qqdmr_re, source, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    if qqdmr:
        return ''.join(qqdmr)
    else:
        return 0
    
df8['qqdmr_text'] = df8.SECFNAME.apply(get_qqdmr)

#### Function for Extraction of text from "Risk Factors" Report

In [30]:
def get_rf(url):
        
    source = requests.get(f'{url}').text.lower()
    rf = re.findall(rf_re, source, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    if rf:
        return ''.join(rf)
    else:
        return 0
    
df8['rf_text'] = df8.SECFNAME.apply(get_rf)

# Output Results of 10 Variables each for mda, qqdmr and rf Reports

In [31]:
df8['mda_positive_score'] = df8.mda_text.apply(positive_score)
df8['mda_negative_score'] = df8.mda_text.apply(negative_score)
df8['mda_polarity_score'] = np.vectorize(polarity_score)(df8['mda_positive_score'],df8['mda_negative_score'])
df8['mda_average_sentence_length'] = df8.mda_text.apply(average_sentence_length)
df8['mda_percentage_of_complex_words'] = df8.mda_text.apply(percentage_complex_word)
df8['mda_fog_index'] = np.vectorize(fog_index)(df8['mda_average_sentence_length'],df8['mda_percentage_of_complex_words'])
df8['mda_complex_word_count']= df8.mda_text.apply(complex_word_count)
df8['mda_word_count'] = df8.mda_text.apply(total_word_count)
df8['mda_uncertainty_score']=df8.mda_text.apply(uncertainty_score)
df8['mda_constraining_score'] = df8.mda_text.apply(constraining_score)


df8['qqdmr_positive_score'] = df8.qqdmr_text.apply(positive_score)
df8['qqdmr_negative_score'] = df8.qqdmr_text.apply(negative_score)
df8['qqdmr_polarity_score'] = np.vectorize(polarity_score)(df8['qqdmr_positive_score'],df8['qqdmr_negative_score'])
df8['qqdmr_average_sentence_length'] = df8.qqdmr_text.apply(average_sentence_length)
df8['qqdmr_percentage_of_complex_words'] = df8.qqdmr_text.apply(percentage_complex_word)
df8['qqdmr_fog_index'] = np.vectorize(fog_index)(df8['qqdmr_average_sentence_length'],df8['qqdmr_percentage_of_complex_words'])
df8['qqdmr_complex_word_count']= df8.qqdmr_text.apply(complex_word_count)
df8['qqdmr_word_count'] = df8.qqdmr_text.apply(total_word_count)
df8['qqdmr_uncertainty_score']=df8.qqdmr_text.apply(uncertainty_score)
df8['qqdmr_constraining_score'] = df8.qqdmr_text.apply(constraining_score)



df8['rf_positive_score'] = df8.rf_text.apply(positive_score)
df8['rf_negative_score'] = df8.rf_text.apply(negative_score)
df8['rf_polarity_score'] = np.vectorize(polarity_score)(df8['rf_positive_score'],df8['rf_negative_score'])
df8['rf_average_sentence_length'] = df8.rf_text.apply(average_sentence_length)
df8['rf_percentage_of_complex_words'] = df8.rf_text.apply(percentage_complex_word)
df8['rf_fog_index'] = np.vectorize(fog_index)(df8['rf_average_sentence_length'],df8['rf_percentage_of_complex_words'])
df8['rf_complex_word_count']= df8.rf_text.apply(complex_word_count)
df8['rf_word_count'] = df8.rf_text.apply(total_word_count)
df8['rf_uncertainty_score']=df8.rf_text.apply(uncertainty_score)
df8['rf_constraining_score'] = df8.rf_text.apply(constraining_score)

# 3       Additional Variables: Positive/Negative and Uncertainty/Constraining Word Proportion

#### Calculating Positive Word Proportion

In [32]:
def positive_word_prop(positivescore,wordcount):
    positive_word_proportion = 0
    if wordcount !=0:
        positive_word_proportion = positivescore / wordcount
        
    return positive_word_proportion

#### Calculating Negative Word Proportion

In [33]:
def negative_word_prop(negativescore,wordcount):
    negative_word_proportion = 0
    if wordcount !=0:
        negative_word_proportion = negativescore / wordcount
        
    return negative_word_proportion

#### Calculating Uncertainty Word Proportion

In [34]:
def uncertain_word_prop(uncertainscore,wordcount):
    uncertain_word_proportion = 0
    if wordcount !=0:
        uncertain_word_proportion = uncertainscore / wordcount
        
    return uncertain_word_proportion

#### Calculating Constraining Word Proportion

In [35]:
def constraining_word_prop(constrainingscore,wordcount):
    constraining_word_proportion = 0
    if wordcount !=0:
        constraining_word_proportion = constrainingscore / wordcount
        
    return constraining_word_proportion

# Output Results of 04 Variables each for mda, qqdmr and rf Reports

In [36]:
df8['mda_positive_word_proportion'] = np.vectorize(positive_word_prop)(df8['mda_positive_score'],df8['mda_word_count'])
df8['mda_negative_word_proportion'] = np.vectorize(negative_word_prop)(df8['mda_negative_score'],df8['mda_word_count'])
df8['mda_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df8['mda_uncertainty_score'],df8['mda_word_count'])
df8['mda_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df8['mda_constraining_score'],df8['mda_word_count'])


df8['qqdmr_positive_word_proportion'] = np.vectorize(positive_word_prop)(df8['qqdmr_positive_score'],df8['qqdmr_word_count'])
df8['qqdmr_negative_word_proportion'] = np.vectorize(negative_word_prop)(df8['qqdmr_negative_score'],df8['qqdmr_word_count'])
df8['qqdmr_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df8['qqdmr_uncertainty_score'],df8['qqdmr_word_count'])
df8['qqdmr_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df8['qqdmr_constraining_score'],df8['qqdmr_word_count'])


df8['rf_positive_word_proportion'] = np.vectorize(positive_word_prop)(df8['rf_positive_score'],df8['rf_word_count'])
df8['rf_negative_word_proportion'] = np.vectorize(negative_word_prop)(df8['rf_negative_score'],df8['rf_word_count'])
df8['rf_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df8['rf_uncertainty_score'],df8['rf_word_count'])
df8['rf_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df8['rf_constraining_score'],df8['rf_word_count'])

# 4	       Additional Variable: Constraining words for whole report

#####  Calculating Constraining Words for Whole Report

In [37]:
df8[["mda_text", "qqdmr_text", "rf_text"]] = df8[["mda_text", "qqdmr_text", "rf_text"]].astype(str) 
def constrain_word_whole(mda_text,qqdmr_text,rf_text):
    wholeDoc = mda_text + qqdmr_text + rf_text
    constrainwordwhole =0
    tokens = tokenizer(wholeDoc)
    for word in tokens:
        if word in constrainDictionary:
            constrainwordwhole +=1
    constrainscorewhole = constrainwordwhole 
    
    return constrainscorewhole

# Output Results of 01 Variable each for mda, qqdmr and rf Reports

In [38]:
df8['constraining_words_whole_report'] = np.vectorize(constrain_word_whole)(df8['mda_text'],df8['qqdmr_text'],df8['rf_text'])

# Display of Output Data Structure for mda, qqdmr and rf Reports

In [39]:
pd.set_option('display.max_columns', 52)

df8

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_text,qqdmr_text,rf_text,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainty_score,qqdmr_constraining_score,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainty_word_proportion,mda_constraining_word_proportion,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainty_word_proportion,qqdmr_constraining_word_proportion,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt,item 7. management's discussion and analysis of financial\n condition and results ...,0,0,20,76,-0.583333,41,0.240543,16.496217,833,3463,30,11,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.005775,0.021946,0.008663,0.003176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-001001.txt,item 2. management's discussion and analysis of financial condition and\n ...,0,0,11,56,-0.671642,46,0.260853,18.504341,691,2649,57,3,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.004153,0.021140,0.021518,0.001133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-000783.txt,0,0,0,0,0,0.000000,1,0.000000,0.400000,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-002145.txt,item 7. management's discussion and analysis of financial 9\n condition and...,0,0,44,149,-0.544041,38,0.255751,15.302301,1712,6694,84,49,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.006573,0.022259,0.012549,0.007320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0000950172-98-001203.txt,0,0,0,0,0,0.000000,1,0.000000,0.400000,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-024804.txt,0,0,0,0,0,0.000000,1,0.000000,0.400000,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-040463.txt,0,0,0,0,0,0.000000,1,0.000000,0.400000,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,https://www.sec.gov/Archives/edgar/data/12239/0001104659-07-041441.txt,0,0,0,0,0,0.000000,1,0.000000,0.400000,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0,0,0.0,1,0.0,0.4,0,1,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,https://www.sec.gov/Archives/edgar/data/122

# 5   Final Output Data Structure

#### Saving of Output Data Structure to '.xlsx' file

In [40]:
writer = pd.ExcelWriter('D:/Black Coffer/Output Data Structure.xlsx', engine='xlsxwriter')
df8.to_excel(writer, sheet_name='Sheet1')
workbook  = writer.book
worksheet = writer.sheets['Sheet1']
writer.save()

#### Conversion of Output Data Structure from '.xlsx' file to '.csv' file

In [41]:
read_file = pd.read_excel (r'D:/Black Coffer/Output Data Structure.xlsx')
read_file.to_csv ('D:/Black Coffer/Output Data Structure.csv', index = None, header = None)